<a href="https://www.kaggle.com/code/burcakaydn/bidding-methods-with-ab-testing?scriptVersionId=168694498" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Analyzing the Effectiveness of Different Bidding Strategies Through A/B Testing

## Business Problem
* Facebook recently introduced a new bidding type called "average bidding" as an alternative to the existing "maximum bidding" type. Our client, bombabomba.com, has decided to test this new feature and wants to conduct an A/B test to determine whether average bidding brings in more conversions than maximum bidding.
* The A/B test has been running for 1 month, and bombabomba.com now expects you to analyze the results of this A/B test.
* Purchase is the ultimate success metric for bombabomba.com. Therefore, the Purchase metric should be focused on for statistical tests.


**"Average bidding"** and **"maximum bidding"** are terms commonly used in the context of online advertising and digital marketing to describe bidding strategies.

**Average Bidding:** In this strategy, the advertiser sets a specific budget and makes an average bid for each click. The advertising platform automatically adjusts the bids based on the frequency of ad impressions and clicks, optimizing them to result in an average cost. This strategy provides advertisers with more control and allows them to manage their budget more effectively.

**Maximum Bidding:** In this strategy, the advertiser sets a maximum bid for each click. The advertising platform automatically adjusts the bids to reach the specified maximum bid. This strategy allows advertisers to bid more aggressively to increase the visibility and likelihood of clicks for their ads.

## Data Story
* This data set contains information about a company's website, including information such as the number of ads seen and clicked by users, as well as revenue information from these ads. There are two separate data sets:
* Control and Test groups. These data sets are on separate pages of the ab_testing.xlsx Excel file. 
* Maximum Bidding is applied to the control group, and Average Bidding is applied to the test group.

    * **Impression:** Number of ad impressions
    * **Click:** Number of clicks on ads
    * **Purchase:** Number of products purchased after clicking on ads
    * **Earning:** Revenue generated after purchasing products


## Project Tasks
### AB Testing (Independent Two-Sample T Test)
1. Formulate hypotheses
2. Assumption Check
- 1. Normality Assumption (shapiro)
- 2. Homogeneity of Variances (levene)
3. Application of the Hypothesis
- 1. Independent two-sample t-test if assumptions are met
- 2. Mann-Whitney U test if assumptions are not met
4. Interpret the results based on the p-value

Note:
- If normality is not satisfied, go directly to number 2. If homogeneity of variances is not satisfied,
pass argument 1 to number 1.
- Before normality examination, outlier examination and correction may be useful.

In [ ]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, \
    pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

## Task 1: Data Preparation and Analysis
### Step 1: 
* Read the data set consisting of control and test groups from the file ab_testing_data.xlsx.
* Assign the control and test group data to separate variables.

In [ ]:
# Get the sheet names
file_path  = pd.ExcelFile('/kaggle/input/ab-testing/ab_testing.xlsx')
sheet_names = file_path.sheet_names
print("Sheet Names in the Excel File:", sheet_names)

In [ ]:
# Read the control and test group data
# Adjust the 'sheet_name' or 'usecols' parameters according to the structure of the Excel file
df_control = pd.read_excel(file_path, sheet_name='Control Group')
df_test = pd.read_excel(file_path, sheet_name='Test Group')

In [ ]:
# Display the control and test group data
print("Control Group Data:")
print(df_control.head())
print("\nTest Group Data:")
print(df_test.head())

### Step 2: Analyze the control and test group data.

In [ ]:
# Statistics of the control group
print("Statistics of the Control Group:")
print(df_control.describe())
# Statistics of the test group
print("\nBasic Statistics of the Test Group:")
print(df_test.describe())

### Step 3: After the analysis process, concatenate the control and test group data using the concat method.


In [ ]:
# Add a column named 'group' and assign group names
df_control['Group'] = 'control'
df_test['Group'] = 'test'

In [ ]:
# Concatenate the control and test group data
df_combined = pd.concat([df_control, df_test], axis=0)

# Show the first few rows of the concatenated data set
print(df_combined.head())
print("\n")
print(df_combined.tail())

## Task 2: Definition of A/B Testing Hypothesis
### Step 1: Define the hypothesis.
        H0: M1 = M2
        H1: M1 != M2
### Step 2: Analyze the purchase (revenue) means for the control and test groups.

In [ ]:
# Mean purchase of the control group
purchase_mean_control = df_control['Purchase'].mean()

# Mean purchase of the test group
purchase_mean_test = df_test['Purchase'].mean()

print("Mean 'Purchase' of the Control Group: %.4f" % purchase_mean_control)
print("Mean 'Purchase' of the Test Group: %.4f" % purchase_mean_test)

## Task 3: Implementation of Hypothesis Testing
AB Testing (Independent Two-Sample T Test)
### Step 1: Perform assumption checks before hypothesis testing. These are Normality Assumption and Homogeneity of Variances.
Test whether the control and test groups conform to the normality assumption based on the Purchase variable

In [ ]:
from scipy.stats import shapiro

# Normality assumption for the control group
test_stat, pvalue = shapiro(df_control['Purchase'])
print('Shapiro Test for Control Group - Purchase: Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

In [ ]:
# Normality assumption for the test group
test_stat, pvalue = shapiro(df_test['Purchase'])
print('Shapiro Test for Test Group - Purchase: Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

## Interpretation
* Since the p-values for both groups are greater than 0.05, the null hypothesis cannot be rejected.
* This means that the Purchase variable values for both groups are normally distributed.
* Since the normality assumption is met, an independent two-sample t-test (parametric test) can be applied.
* However, before making a final decision, a test of homogeneity of variances should also be performed.

In [ ]:
from scipy.stats import levene

# Homogeneity of variances test
test_stat, pvalue = levene(df_control['Purchase'], df_test['Purchase'])
print('Levene Test for Homogeneity of Variances: Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

## Interpretation
* Since the p-value is greater than 0.05, there is no statistically significant difference
* between the variances. Therefore, the assumption of homogeneity of variances is satisfied.
* Thus, an independent two-sample t-test (parametric test) can be applied.

### Step 2: Select the appropriate test based on the results of the Normality Assumption and Homogeneity of Variances.
### Independent Two-Sample T Test (When Assumptions are Satisfied)

In [ ]:
from scipy.stats import ttest_ind

test_stat, pvalue = ttest_ind(df_control['Purchase'], df_test['Purchase'], equal_var=True)
print('Independent Two-Sample T Test: Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

## Step 3: Interpret the test results based on the obtained p-value.
## Interpretation
* Since the p-value is greater than 0.05, there is no statistically significant difference between the purchase means of the control and test groups. 
* Therefore, it can be concluded that there is no statistically significant difference in conversion between the control group using maximum bidding and the test group using average bidding. 
* Thus, there is currently no reason for the client to prefer one bidding method over the other. 
* However, it may be advisable to conduct a more comprehensive analysis taking into account the client's long-term performance and other factors.
